# Camenisch-Lysyanskaya Signatures

This contains my notes on learning Camenisch-Lysyanskaya signatures from scratch (I knew very little about ZKPs). The references I used are:

* [Direct Anonymous Attestation Explained](https://pdfs.semanticscholar.org/4cc6/257c4b2d649a09b08c1789dcaf3aeb3c1fda.pdf)

* [Modular Cryptographic Protocol Design](https://camenisch.org/jan/presentations/2018-09-11-Modular-Protocol-Design-ISC-Surrey.pdf)

* [Zero Knowledge Proofs: An illustrated primer, Part 2](https://blog.cryptographyengineering.com/2017/01/21/zero-knowledge-proofs-an-illustrated-primer-part-2/)

# Schnorr’s Identification Scheme

CL Signatures require proofs of knowledge of some relationship between discrete logarithm. The first that is discussed is Schnorr's Identification scheme.

Schnorr's Identification scheme is a proof of knowledge of some discrete logarithm of `y = g^x`. `x` is the secret known the prover, and `y` is known to both parties. This scheme allows the prover to demonstrate knowledge of `x` without leaking any information about `x`.

In [31]:
p = 2*251 + 1 # safe prime p
Zp = Zmod(p)
g = Zp.multiplicative_generator()
q = g.multiplicative_order()
Zq = Zmod(q)
k = 10

class SchnorrProver(object):
    
    def __init__(self):
        self.secret_key = Zq.random_element()
        self.public_key = g^self.secret_key
    
    def commit(self):
        self.r = Zq.random_element()
        return g^self.r
    
    def respond(self, challenge):
        return self.r - challenge*self.secret_key

class SchnorrVerifier(object):
    
    def __init__(self, prover_public_key):
        self.prover_public_key = prover_public_key
    
    def challenge(self, commitment):
        self.commitment = commitment
        self.challenge_number = randint(1, 2^k)
        return self.challenge_number
    
    def check(self, response):
        check_value = g^response*self.prover_public_key^self.challenge_number
        return self.commitment == check_value

def run_protocol(iterations=32):
    for _ in xrange(iterations):
        prover = SchnorrProver()
        verifier = SchnorrVerifier(prover.public_key)
        t = prover.commit()
        c = verifier.challenge(t)
        s = prover.respond(c)
        assert(verifier.check(s))
    
run_protocol()

#  Proving Knowledge of a Representation

An extension of the Schnoor Identification protocol involves proving knowledge of multiple discrete logs of many bases in a group. So, instead of proving knowledge of just `x` s.t. `y = g^x`, we'd prove knowledge of `x1, x2, ..., xl` s.t. `y = g1^x1 * g2^x2 * ... * gl^xl`.

This generalization and the simpler identification protocol are almost identitical.

In [32]:
p = 2*251 + 1 # safe prime p
Zp = Zmod(p)
g = Zp.multiplicative_generator()
q = g.multiplicative_order()
Zq = Zmod(q)
k = 10
l = 8

# construct `l` bases in the subgroup of order q
gs = []
while len(gs) != l:
    x = randint(1, q)
    if gcd(x, p-1) == 1:
        gx = g^x
        assert(gx.multiplicative_order() == q)
        gs.append(g^x)
        
class GeneralSchnorrProver(object):
    
    def __init__(self):
        self.secret_keys = [Zq.random_element() for _ in range(l)]
        public_key = Zp(1)
        for (g, s) in zip(gs, self.secret_keys):
            public_key *= g^s
        self.public_key = public_key
    
    def commit(self):
        self.rs = [Zq.random_element() for _ in range(l)]
        t = Zp(1)
        for (g, r) in zip(gs, self.rs):
            t *= g^r
        return t
    
    def respond(self, challenge):
        return [r - challenge*s 
                for (r, s) in zip(self.rs, self.secret_keys)]

class GeneralSchnorrVerifier(object):
    
    def __init__(self, prover_public_key):
        self.prover_public_key = prover_public_key
    
    def challenge(self, commitment):
        self.commitment = commitment
        self.challenge_number = randint(1, 2^k)
        return self.challenge_number
    
    def check(self, response):
        check_value = self.prover_public_key^self.challenge_number
        for (g, r) in zip(gs, response):
            check_value *= g^r
        return self.commitment == check_value

def run_protocol(iterations=32):
    for _ in xrange(iterations):
        prover = GeneralSchnorrProver()
        verifier = GeneralSchnorrVerifier(prover.public_key)
        t = prover.commit()
        c = verifier.challenge(t)
        s = prover.respond(c)
        assert(verifier.check(s))

run_protocol()

# Combining Different Proof Protocols

A further generalization of identification protocols involves running multiple proof protocols in parrallel. Imagine you were proving knowledge of two secrets `s1` and `s2` s.t. `x = g^s1` and `y = h^s2` where `g` and `h` are both members of the same group. To run these protocols in parallel, you can do what the paper describes:


> First, the prover computes and sends to the verifier the commitment messages of both protocols at the same time. Next, the verifier chooses and sends back a single challenge message, i.e., the verifier chooses the same challenge message for both protocols. Finally, the verifier will accept the overall protocol only if the verification equations of both (sub-)protocols hold.


In other words, you can use a single _verifier_ challenge across protocols in parallel and things don't seem to fall apart (IDK how general this claim is).

# Aside: "standard rewinding techniques"

In "Direct Anonymous Attestation Explained" the author mentions a proof technique referred to as "standard rewinding techniques":

> Let us explain why the verifier should be convinced that $log_{h}(z)$ equals the first element in the representation of $y$ w.r.t. $g$ and $h$. Using standard rewinding techniques, one can obtain from a successful prover commitment and response messages for different challenge messages but the same commitment messages...

This technique is demonstrated in "Zero Knowledge Proofs: An illustrated primer, Part 2" as a method for proving the soundness properties of a zero-knowledge proof system.

Recall that a ZKP is considered sound if "The Prover can only convince the Verifier if the statement is true". Or put another way: "If [the Prover] successfully convinces [the Verifier], then [the Prover] must know the [secrets]".

The technique used for formally proving soundness is to prove that there exists an algorithm for the verifier to extract the prover's secret if the prover demonstrates their proof. The blog post calls this algorithm the "Extractor". This algorithm under "normal" circumstances shouldn't be possible (otherwise the protocol would just be trivially broken!). However, the proof used in this blog post relies on the "Extractor" being able to stop a ZKP protocol run mid-protocol and "rewind" the prover's state. The extractor defined in the "Illustrated Primer, Part 2" blog post for Schnorr's Identification scheme is very similar to the proofs used in "Direct Anonymous Attestation Explained"!